# Capstone : IT Ticket Classification [SEPT SUN GRP 4B]

![1_yK5G9nHmOD-wrJSRSvPEpw.jpeg](attachment:1_yK5G9nHmOD-wrJSRSvPEpw.jpeg)

# Aim: Automatic Ticket Assignment [Part 5/6]


Build a classifier that can classify the tickets by analyzing text. Classify incidents to right functional groups can help organizations to reduce the resolving time of the issue and can focus on more productive tasks.

## 5/6: Model Building with Machine Learning Models

- Building a model architecture which can classify
- Trying different model architectures by researching state of the art for similar tasks
- Train the model
- To deal with large training time, save the weights so that we can use them when training the model for the second time without starting from scratch.

# Import Libraries

In [2]:
import pandas as pd 
import numpy as np 
import re 
import sys
import seaborn as sns
import matplotlib.pyplot as plt
import random
import time

import warnings
import pandas.testing as tm
import spacy  
#import langid 
import nltk

import re, string
from wordcloud import WordCloud
from pprint import pprint

from sklearn.utils import resample
from sklearn import preprocessing

from nltk.corpus import stopwords
from nltk.classify.textcat import TextCat  
from dateutil import parser
from nltk.tokenize import word_tokenize
#from langdetect import detect
from textblob import TextBlob
#from googletrans import Translator

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models.ldamodel import LdaModel
from gensim.models import CoherenceModel

#import pyLDAvis
#import pyLDAvis.gensim 

from sklearn.decomposition import LatentDirichletAllocation

from string import punctuation

#import enchant
#from enchant.checker import SpellChecker

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
    
from sklearn.feature_extraction.text import CountVectorizer
import plotly.express as px

from gensim.models import Word2Vec
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten, Bidirectional, GlobalMaxPool1D,GRU,Conv1D,MaxPooling1D
from tensorflow.keras.models import Model, Sequential
import tensorflow as tf
from sklearn import metrics
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

from scipy import spatial
from sklearn.manifold import TSNE

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB

import logging, warnings
from PIL import Image

##--------------------##

if sys.version_info >= (3, 3) :
    import collections.abc as collections_abc
else:
    import collections as collections_abc
    
from collections import Counter
from collections import OrderedDict
import operator

##-------------------##

plt.style.use('ggplot')
%matplotlib inline

#warnings.filterwarnings("ignore",category=DeprecationWarning)
#warnings.filterwarnings(action='once')
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

In [3]:
np.__version__ , pd.__version__,tf.__version__, print(sys.version_info)

sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)


('1.18.5', '1.0.5', '2.3.0', None)

In [4]:
"""
Download required NLTK stopwords corpus if it has not already been downloaded.
Download required NLTK corpora if they have not already been downloaded.
Download the NLTK averaged perceptron tagger that is required for this algorithm to run only if the corpora has not already been downloaded.
Download Punkt Sentence Tokenizer. This tokenizer divides a text into a list of sentences, by using an unsupervised algorithm to build a model for abbreviation words, collocations, and words that start sentences
"""
nltk.download('wordnet', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True) 

True

# Data Loading

In [5]:
#"""
from google.colab import drive
drive.mount('/content/drive')

project_path = '/content/drive/My Drive/Colab Notebooks/GLCapstone/'
file_name ='itsupportdatacleaned_3.csv'

#"""
#project_path = 'C:/MyDrive/Drive2/PersonalDrive/MachineLearning/GreatLearning/CapstoneProject/Final-Deck/'
#file_name ='itsupportdatacleaned_3.csv'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [6]:
data=pd.read_csv(project_path+file_name,encoding=sys.getfilesystemencoding()) 
data.head(2).T

,0,1
Short description,collaboration_platform site is not opening,account locked.
Description,collaboration platform site open collaboration...,account lock account lock
Caller,anpocezt qturbxsg,bkyphsgq cfyksehu
Assignment group,GRP_0,GRP_0
New_Assignment_Groups,0,0
Text_length,42,15
Dominant_Topic,0,0
Topic_Perc_Contrib,0.5092,0.9104


In [7]:
data.dropna(subset=[data.columns[1]], inplace=True)

In [8]:
data.head(2).T

,0,1
Short description,collaboration_platform site is not opening,account locked.
Description,collaboration platform site open collaboration...,account lock account lock
Caller,anpocezt qturbxsg,bkyphsgq cfyksehu
Assignment group,GRP_0,GRP_0
New_Assignment_Groups,0,0
Text_length,42,15
Dominant_Topic,0,0
Topic_Perc_Contrib,0.5092,0.9104


In [9]:
data.shape, data.columns[1]

((46991, 8), 'Description')

In [10]:
data_3_resampled = data.copy()

In [11]:
def wordTokenizer(dataframe):
    tokenizer = Tokenizer(num_words=numWords,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',lower=True,split=' ', char_level=False)
    tokenizer.fit_on_texts(dataframe)
    dataframe = tokenizer.texts_to_sequences(dataframe)
    return tokenizer,dataframe

In [12]:
data_3_resampled.reset_index(drop=True, inplace=True)

data_3_resampled.columns[1],data_3_resampled.columns[4]

maxlen = 150
numWords = 9000

# ML Model 1 Training :  Random Forest Classifier (Dataset 3)

Note: Tree based models work by learning in hierarchical manner. Random forests is a supervised learning algorithm which can be used both for classification as well as regression. It is also the most flexible and easy to use algorithm. It creates decision trees on randomly selected data samples, gets prediction from each tree and selects the best solution by means of voting. It also provides a pretty good indicator of the feature importance. 

In [13]:
tokenizer,X = wordTokenizer(data_3_resampled[data_3_resampled.columns[1]])

y = np.asarray(data_3_resampled[data_3_resampled.columns[4]])
X = pad_sequences(X, maxlen = maxlen)

print("Number of Samples:", len(X))
print("Number of Labels: ", len(y))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)


print("Number of Training Samples:", len(X_train))
print("Number of Validation Samples:", len(X_test))

data_3_resampled_RFC=RandomForestClassifier(n_estimators=100)

start = time.time()
data_3_resampled_RFC.fit(X_train,y_train)
time.sleep(1)
end = time.time()
print(f"Time Taken To Train RFC on dataset 3 : {end - start}")


y_pred=data_3_resampled_RFC.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

print(classification_report(y_test, data_3_resampled_RFC.predict(X_test), digits=4))

results_ML_Models = pd.DataFrame()
results_data_3_resampled_RFC = pd.DataFrame({'Model':['Random Forest'], 'accuracy': [metrics.accuracy_score(y_test, y_pred)]},index={'1'})
results_ML_Models = pd.concat([results_ML_Models, results_data_3_resampled_RFC])
results_ML_Models



Number of Samples: 46991
Number of Labels:  46991
Number of Training Samples: 37592
Number of Validation Samples: 9399
Time Taken To Train RFC on dataset 3 : 9.933714628219604
Accuracy: 0.9494627087988083
              precision    recall  f1-score   support

           0     0.7928    0.6241    0.6984       141
           1     0.8881    1.0000    0.9407       119
           2     0.9821    0.8271    0.8980       133
           3     1.0000    1.0000    1.0000       128
           4     0.9091    0.8759    0.8922       137
           5     0.9365    0.9593    0.9478       123
           6     0.9746    0.9746    0.9746       118
           7     1.0000    1.0000    1.0000       137
           8     0.9786    1.0000    0.9892       137
           9     0.9932    1.0000    0.9966       145
          10     0.9508    0.9431    0.9469       123
          11     0.9350    0.9504    0.9426       121
          12     0.9286    0.9369    0.9327       111
          13     0.9916    1.0000    0

,Model,accuracy
1,Random Forest,0.949463


#  ML Model 2 : Random Forest Classifier (Weighted, Dataset 3)

Note: We have received good accuracy using RFC. Also there is not much difference between with and without assigning the class weights which we plan to explore further.


In [14]:
tokenizer,X = wordTokenizer(data_3_resampled[data_3_resampled.columns[1]])

y = np.asarray(data_3_resampled[data_3_resampled.columns[4]])
X = pad_sequences(X, maxlen = maxlen)

print("Number of Samples:", len(X))
print("Number of Labels: ", len(y))


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

print("Number of Training Samples:", len(X_train))
print("Number of Validation Samples:", len(X_test))

data_3_resampled_RFC_W = RandomForestClassifier(n_estimators=100, criterion = 'entropy', class_weight='balanced') # Class Weights

start = time.time()
data_3_resampled_RFC_W.fit(X_train,y_train)
time.sleep(1)
end = time.time()
print(f"Time Taken To Train RFC Weighted on dataset 3: {end - start}")

y_pred=data_3_resampled_RFC_W.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

print(classification_report(y_test, data_3_resampled_RFC_W.predict(X_test), digits=4))

#results_ML_Models = pd.DataFrame()
results_data_3_resampled_RFC_W = pd.DataFrame({'Model':['Random Forest Classifier - Weighted'], 'accuracy': [metrics.accuracy_score(y_test, y_pred)]},index={'1'})
results_ML_Models = pd.concat([results_ML_Models, results_data_3_resampled_RFC_W])
results_ML_Models


Number of Samples: 46991
Number of Labels:  46991
Number of Training Samples: 37592
Number of Validation Samples: 9399
Time Taken To Train RFC Weighted on dataset 3: 17.530489683151245
Accuracy: 0.9497818916906053
              precision    recall  f1-score   support

           0     0.8426    0.6454    0.7309       141
           1     0.8881    1.0000    0.9407       119
           2     0.9649    0.8271    0.8907       133
           3     1.0000    1.0000    1.0000       128
           4     0.9453    0.8832    0.9132       137
           5     0.9219    0.9593    0.9402       123
           6     0.9829    0.9746    0.9787       118
           7     1.0000    1.0000    1.0000       137
           8     0.9856    1.0000    0.9928       137
           9     0.9932    1.0000    0.9966       145
          10     0.9748    0.9431    0.9587       123
          11     0.9504    0.9504    0.9504       121
          12     0.8595    0.9369    0.8966       111
          13     1.0000    1.

,Model,accuracy
1,Random Forest,0.949463
1,Random Forest Classifier - Weighted,0.949782


#  ML Model 3 : Adaboost (Dataset 3)

In [15]:
tokenizer,X = wordTokenizer(data_3_resampled[data_3_resampled.columns[1]])

y = np.asarray(data_3_resampled[data_3_resampled.columns[4]])
X = pad_sequences(X, maxlen = maxlen)

print("Number of Samples:", len(X))
print("Number of Labels: ", len(y))


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

print("Number of Training Samples:", len(X_train))
print("Number of Validation Samples:", len(X_test))


from sklearn.ensemble import AdaBoostClassifier

data_3_resampled_Ada = AdaBoostClassifier(n_estimators=100, learning_rate=0.1, algorithm='SAMME.R', random_state=22) 

start = time.time()
data_3_resampled_Ada.fit(X_train,y_train)
time.sleep(1)
end = time.time()
print(f"Time Taken To Train Adaboost on dataset 3 : {end - start}")

y_pred=data_3_resampled_Ada.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

print(classification_report(y_test, data_3_resampled_Ada.predict(X_test), digits=4))

#results_ML_Models = pd.DataFrame()
results_data_3_resampled_Ada= pd.DataFrame({'Model':['Adaboost Classifier'], 'accuracy': [metrics.accuracy_score(y_test, y_pred)]},index={'1'})
results_ML_Models = pd.concat([results_ML_Models, results_data_3_resampled_Ada])
results_ML_Models


Number of Samples: 46991
Number of Labels:  46991
Number of Training Samples: 37592
Number of Validation Samples: 9399
Time Taken To Train Adaboost on dataset 3 : 25.727181434631348
Accuracy: 0.08234918608362592
              precision    recall  f1-score   support

           0     0.0412    0.0709    0.0521       141
           1     0.0000    0.0000    0.0000       119
           2     0.7556    0.2556    0.3820       133
           3     0.0000    0.0000    0.0000       128
           4     0.0452    0.0657    0.0536       137
           5     0.0351    0.0163    0.0222       123
           6     0.0169    0.0593    0.0263       118
           7     0.0178    0.0584    0.0273       137
           8     0.0000    0.0000    0.0000       137
           9     0.5833    0.0483    0.0892       145
          10     0.0784    0.0325    0.0460       123
          11     0.0000    0.0000    0.0000       121
          12     0.0195    0.0450    0.0272       111
          13     0.0259    0.02

,Model,accuracy
1,Random Forest,0.949463
1,Random Forest Classifier - Weighted,0.949782
1,Adaboost Classifier,0.082349


#  ML Model 4 : Bagging (Dataset 3)

In [16]:
tokenizer,X = wordTokenizer(data_3_resampled[data_3_resampled.columns[1]])

y = np.asarray(data_3_resampled[data_3_resampled.columns[4]])
X = pad_sequences(X, maxlen = maxlen)

print("Number of Samples:", len(X))
print("Number of Labels: ", len(y))


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

print("Number of Training Samples:", len(X_train))
print("Number of Validation Samples:", len(X_test))


from sklearn.ensemble import BaggingClassifier

data_3_resampled_Bagg= BaggingClassifier(n_estimators=200, oob_score=True, bootstrap=True, random_state=22) 

start = time.time()
data_3_resampled_Bagg.fit(X_train,y_train)
time.sleep(1)
end = time.time()
print(f"Time Taken To Train Bagging on dataset 3 : {end - start}")

y_pred=data_3_resampled_Bagg.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

print(classification_report(y_test, data_3_resampled_Bagg.predict(X_test), digits=4))

#results_ML_Models = pd.DataFrame()
results_data_3_resampled_Bagg = pd.DataFrame({'Model':['Bagging Classifier'], 'accuracy': [metrics.accuracy_score(y_test, y_pred)]},index={'1'})
results_ML_Models = pd.concat([results_ML_Models, results_data_3_resampled_Bagg])
results_ML_Models

Number of Samples: 46991
Number of Labels:  46991
Number of Training Samples: 37592
Number of Validation Samples: 9399
Time Taken To Train Bagging on dataset 3 : 222.9662311077118
Accuracy: 0.947015639961698
              precision    recall  f1-score   support

           0     0.8608    0.4823    0.6182       141
           1     0.8815    1.0000    0.9370       119
           2     0.9735    0.8271    0.8943       133
           3     0.9846    1.0000    0.9922       128
           4     0.9370    0.8686    0.9015       137
           5     0.9516    0.9593    0.9555       123
           6     0.9829    0.9746    0.9787       118
           7     0.9786    1.0000    0.9892       137
           8     0.9580    1.0000    0.9786       137
           9     0.9667    1.0000    0.9831       145
          10     0.9831    0.9431    0.9627       123
          11     0.8779    0.9504    0.9127       121
          12     0.9444    0.9189    0.9315       111
          13     1.0000    1.0000  

,Model,accuracy
1,Random Forest,0.949463
1,Random Forest Classifier - Weighted,0.949782
1,Adaboost Classifier,0.082349
1,Bagging Classifier,0.947016


#  ML Model 5 : GradientBoost (Dataset 3)

In [17]:
tokenizer,X = wordTokenizer(data_3_resampled[data_3_resampled.columns[1]])

y = np.asarray(data_3_resampled[data_3_resampled.columns[4]])
X = pad_sequences(X, maxlen = maxlen)

print("Number of Samples:", len(X))
print("Number of Labels: ", len(y))


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

print("Number of Training Samples:", len(X_train))
print("Number of Validation Samples:", len(X_test))


from sklearn.ensemble import GradientBoostingClassifier

data_3_resampled_gd= GradientBoostingClassifier(n_estimators=100, learning_rate = 0.05, random_state=22) 

start = time.time()
data_3_resampled_gd.fit(X_train,y_train)
time.sleep(1)
end = time.time()
print(f"Time Taken To Train Bagging on dataset 3 : {end - start}")

y_pred=data_3_resampled_gd.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

print(classification_report(y_test, data_3_resampled_gd.predict(X_test), digits=4))

#results_ML_Models = pd.DataFrame()
results_data_3_resampled_gd= pd.DataFrame({'Model':['GradientBoost Classifier'], 'accuracy': [metrics.accuracy_score(y_test, y_pred)]},index={'1'})
results_ML_Models  = pd.concat([results_ML_Models, results_data_3_resampled_gd])
results_ML_Models

Number of Samples: 46991
Number of Labels:  46991
Number of Training Samples: 37592
Number of Validation Samples: 9399
Time Taken To Train Bagging on dataset 3 : 2589.002761363983
Accuracy: 0.8744547292265135
              precision    recall  f1-score   support

           0     0.5281    0.3333    0.4087       141
           1     0.8380    1.0000    0.9119       119
           2     0.9263    0.6617    0.7719       133
           3     0.8312    1.0000    0.9078       128
           4     0.8000    0.4964    0.6126       137
           5     0.8500    0.6911    0.7623       123
           6     0.7218    0.8136    0.7649       118
           7     0.9041    0.9635    0.9329       137
           8     0.7973    0.8613    0.8281       137
           9     0.9539    1.0000    0.9764       145
          10     0.7176    0.7642    0.7402       123
          11     0.6829    0.4628    0.5517       121
          12     0.7901    0.5766    0.6667       111
          13     0.9048    0.9661 

,Model,accuracy
1,Random Forest,0.949463
1,Random Forest Classifier - Weighted,0.949782
1,Adaboost Classifier,0.082349
1,Bagging Classifier,0.947016
1,GradientBoost Classifier,0.874455


In [19]:
tokenizer,X = wordTokenizer(data_3_resampled[data_3_resampled.columns[1]])

y = np.asarray(data_3_resampled[data_3_resampled.columns[4]])
X = pad_sequences(X, maxlen = maxlen)

print("Number of Samples:", len(X))
print("Number of Labels: ", len(y))


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

print("Number of Training Samples:", len(X_train))
print("Number of Validation Samples:", len(X_test))

from xgboost import XGBClassifier

data_3_resampled_xg = XGBClassifier() 

start = time.time()
data_3_resampled_xg.fit(X_train,y_train)
time.sleep(1)
end = time.time()
print(f"Time Taken To Train XGBoost on dataset 3 : {end - start}")

y_pred=data_3_resampled_xg.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

print(classification_report(y_test, data_3_resampled_xg.predict(X_test), digits=4))

results_data_3_resampled_xg= pd.DataFrame({'Model':['XGBoost Classifier'], 'accuracy': [metrics.accuracy_score(y_test, y_pred)]},index={'1'})
results_ML_Models  = pd.concat([results_ML_Models, results_data_3_resampled_xg])
results_ML_Models

Number of Samples: 46991
Number of Labels:  46991
Number of Training Samples: 37592
Number of Validation Samples: 9399
Time Taken To Train XGBoost on dataset 3 : 881.4950704574585
Accuracy: 0.8822215129269071
              precision    recall  f1-score   support

           0     0.5233    0.3191    0.3965       141
           1     0.8156    0.9664    0.8846       119
           2     0.9186    0.5940    0.7215       133
           3     0.9275    1.0000    0.9624       128
           4     0.8242    0.5474    0.6579       137
           5     0.8165    0.7236    0.7672       123
           6     0.7857    0.7458    0.7652       118
           7     0.8618    0.9562    0.9066       137
           8     0.8880    0.8102    0.8473       137
           9     0.9603    1.0000    0.9797       145
          10     0.8362    0.7886    0.8117       123
          11     0.6444    0.4793    0.5498       121
          12     0.7423    0.6486    0.6923       111
          13     0.8837    0.9661 

,Model,accuracy
1,Random Forest,0.949463
1,Random Forest Classifier - Weighted,0.949782
1,Adaboost Classifier,0.082349
1,Bagging Classifier,0.947016
1,GradientBoost Classifier,0.874455
1,XGBoost Classifier,0.882222


In [20]:
#!pip install catboost

     |████████████████████████████████| 66.1MB 64kB/s 


In [21]:
tokenizer,X = wordTokenizer(data_3_resampled[data_3_resampled.columns[1]])

y = np.asarray(data_3_resampled[data_3_resampled.columns[4]])
X = pad_sequences(X, maxlen = maxlen)

print("Number of Samples:", len(X))
print("Number of Labels: ", len(y))


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

print("Number of Training Samples:", len(X_train))
print("Number of Validation Samples:", len(X_test))


from catboost import CatBoostClassifier

data_3_resampled_cb = CatBoostClassifier() 

start = time.time()
data_3_resampled_cb.fit(X_train,y_train)
time.sleep(1)
end = time.time()
print(f"Time Taken To Train CatBoost on dataset 3 : {end - start}")

y_pred=data_3_resampled_cb.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

print(classification_report(y_test, data_3_resampled_cb.predict(X_test), digits=4))

results_data_3_resampled_cb= pd.DataFrame({'Model':['CatBoost Classifier'], 'accuracy': [metrics.accuracy_score(y_test, y_pred)]},index={'1'})
results_ML_Models  = pd.concat([results_ML_Models, results_data_3_resampled_cb])
results_ML_Models

Number of Samples: 46991
Number of Labels:  46991
Number of Training Samples: 37592
Number of Validation Samples: 9399
Learning rate set to 0.095204
0:	learn: 3.8768763	total: 7.66s	remaining: 2h 7m 30s
1:	learn: 3.6623057	total: 13.6s	remaining: 1h 53m 2s
2:	learn: 3.3778772	total: 19.5s	remaining: 1h 47m 58s
3:	learn: 3.2286763	total: 25.4s	remaining: 1h 45m 25s
4:	learn: 3.0328385	total: 31.3s	remaining: 1h 43m 47s
5:	learn: 2.9334516	total: 37.2s	remaining: 1h 42m 47s
6:	learn: 2.8381189	total: 43.1s	remaining: 1h 41m 57s
7:	learn: 2.7431585	total: 49.2s	remaining: 1h 41m 37s
8:	learn: 2.6572968	total: 55.2s	remaining: 1h 41m 13s
9:	learn: 2.5576882	total: 1m 1s	remaining: 1h 40m 44s
10:	learn: 2.4737256	total: 1m 7s	remaining: 1h 40m 25s
11:	learn: 2.4365443	total: 1m 12s	remaining: 1h 40m 9s
12:	learn: 2.3918789	total: 1m 18s	remaining: 1h 39m 53s
13:	learn: 2.3120466	total: 1m 24s	remaining: 1h 39m 34s
14:	learn: 2.2532903	total: 1m 30s	remaining: 1h 39m 18s
15:	learn: 2.1968676

,Model,accuracy
1,Random Forest,0.949463
1,Random Forest Classifier - Weighted,0.949782
1,Adaboost Classifier,0.082349
1,Bagging Classifier,0.947016
1,GradientBoost Classifier,0.874455
1,XGBoost Classifier,0.882222
1,CatBoost Classifier,0.931908


# ML Model 6 :  SVM & SGD Classifier (Dataset 3)

Note: A support vector machine (SVM) is a supervised machine learning model that uses classification algorithms for two-group classification problems. After giving an SVM model sets of labeled training data for each category, they’re able to categorize the new text. It is a fast and dependable classification algorithm that performs very well with a limited amount of data. A popular algorithm for this technique is penalized SVM. 

In [22]:
tokenizer, X = wordTokenizer(data_3_resampled[data_3_resampled.columns[1]])

y = np.asarray(data_3_resampled[data_3_resampled.columns[4]])
X = pad_sequences(X, maxlen = maxlen)

print("Number of Samples:", len(X))
print("Number of Labels: ", len(y))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

print("Number of Training Samples:", len(X_train))
print("Number of Validation Samples:", len(X_test))

data_3_resampled_SVC = svm.SVC(kernel='rbf')


Cs = [0.01 ]  # [0.01, 0.1 ] #Cs = [0.01, 0.1, 1, 10]
gammas = [0.001 ] # [0.001, 0.01 ] #gammas = [0.001, 0.01, 0.1, 1]
param_grid = {'C': Cs, 'gamma' : gammas}

#start = time.time()
#grid_search = GridSearchCV(data_3_resampled_SVC, param_grid)
#grid_search.fit(X_train,y_train)
#time.sleep(1)
#end = time.time()
#print(f"Time Taken To Get Best Parameters For SVC on dataset 3 dataset with Grid Search: {end - start}")

#data_3_resampled_SVC = svm.SVC(**grid_search.best_params_)
data_3_resampled_SVC = svm.SVC(kernel='rbf', C=.01, gamma=.001)

start = time.time()
data_3_resampled_SVC.fit(X_train, y_train)
time.sleep(1)
end = time.time()
print(f"Time Taken To Train SVC on dataset 3 : {end - start}")

y_pred_SVC = data_3_resampled_SVC.predict(X_test) #predictig using SVC Classifier

print("Accuracy:",metrics.accuracy_score(y_test, y_pred_SVC))

report = classification_report(y_test, data_3_resampled_SVC.predict(X_test), digits=4)

print(classification_report(y_test, data_3_resampled_SVC.predict(X_test), digits=4))

results_data_3_resampled_SVC = pd.DataFrame({'Model':['SVM Classifier'], 'accuracy': [metrics.accuracy_score(y_test, y_pred_SVC)]},index={'2'})
results_ML_Models = pd.concat([results_ML_Models, results_data_3_resampled_SVC])
results_ML_Models

Number of Samples: 46991
Number of Labels:  46991
Number of Training Samples: 37592
Number of Validation Samples: 9399
Time Taken To Train SVC on dataset 3 : 636.471348285675
Accuracy: 0.27981700180870306
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000       141
           1     0.0000    0.0000    0.0000       119
           2     0.9444    0.2556    0.4024       133
           3     0.0000    0.0000    0.0000       128
           4     0.0000    0.0000    0.0000       137
           5     0.0000    0.0000    0.0000       123
           6     0.0000    0.0000    0.0000       118
           7     0.0000    0.0000    0.0000       137
           8     0.0000    0.0000    0.0000       137
           9     0.0000    0.0000    0.0000       145
          10     0.0000    0.0000    0.0000       123
          11     0.0000    0.0000    0.0000       121
          12     0.0000    0.0000    0.0000       111
          13     0.0000    0.0000    0

,Model,accuracy
1,Random Forest,0.949463
1,Random Forest Classifier - Weighted,0.949782
1,Adaboost Classifier,0.082349
1,Bagging Classifier,0.947016
1,GradientBoost Classifier,0.874455
1,XGBoost Classifier,0.882222
1,CatBoost Classifier,0.931908
2,SVM Classifier,0.279817


In [23]:
tokenizer, X = wordTokenizer(data_3_resampled[data_3_resampled.columns[1]])

y = np.asarray(data_3_resampled[data_3_resampled.columns[4]])
X = pad_sequences(X, maxlen = maxlen)

print("Number of Samples:", len(X))
print("Number of Labels: ", len(y))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

print("Number of Training Samples:", len(X_train))
print("Number of Validation Samples:", len(X_test))

data_3_resampled_SGD = SGDClassifier()

start = time.time()
data_3_resampled_SGD.fit(X_train, y_train)
time.sleep(1)
end = time.time()
print(f"Time Taken To Train SGD on dataset 3 : {end - start}")

y_pred_SGD = data_3_resampled_SGD.predict(X_test) #predicting using SGD Classifier

#accuracy_score(y_test,y_pred_SGD) #accuracy score of two models
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_SGD))
print(classification_report(y_test, data_3_resampled_SGD.predict(X_test), digits=4))

results_data_3_resampled_SGD = pd.DataFrame({'Model':['SGD Classifier'], 'accuracy': [metrics.accuracy_score(y_test, y_pred_SGD)]},index={'2'})
results_ML_Models = pd.concat([results_ML_Models, results_data_3_resampled_SGD])
results_ML_Models

Number of Samples: 46991
Number of Labels:  46991
Number of Training Samples: 37592
Number of Validation Samples: 9399
Time Taken To Train SGD on dataset 3 : 142.51342153549194
Accuracy: 0.2743908926481541
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000       141
           1     0.0727    0.4034    0.1232       119
           2     0.2308    0.0226    0.0411       133
           3     0.0742    0.2656    0.1160       128
           4     0.3333    0.0219    0.0411       137
           5     0.2817    0.1626    0.2062       123
           6     0.0447    0.0678    0.0539       118
           7     0.0510    0.2701    0.0857       137
           8     0.4000    0.0584    0.1019       137
           9     0.0455    0.0483    0.0468       145
          10     0.0703    0.1463    0.0950       123
          11     0.0385    0.0165    0.0231       121
          12     0.2353    0.1081    0.1481       111
          13     0.2568    0.3983    

,Model,accuracy
1,Random Forest,0.949463
1,Random Forest Classifier - Weighted,0.949782
1,Adaboost Classifier,0.082349
1,Bagging Classifier,0.947016
1,GradientBoost Classifier,0.874455
1,XGBoost Classifier,0.882222
1,CatBoost Classifier,0.931908
2,SVM Classifier,0.279817
2,SGD Classifier,0.274391


# ML Model 7: Naive Bayes (Dataset 3)

In [24]:
tokenizer,X = wordTokenizer(data_3_resampled[data_3_resampled.columns[1]])

y = np.asarray(data_3_resampled[data_3_resampled.columns[4]])
X = pad_sequences(X, maxlen = maxlen)

print("Number of Samples:", len(X))
print("Number of Labels: ", len(y))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

print("Number of Training Samples:", len(X_train))
print("Number of Validation Samples:", len(X_test))

data_3_resampled_NB = MultinomialNB().fit(X_train, y_train)

y_pred_NB = data_3_resampled_NB.predict(X_test)

accuracy_score(y_test,y_pred_NB)

report = classification_report(y_test, data_3_resampled_NB.predict(X_test), digits=4)

print(classification_report(y_test, data_3_resampled_NB.predict(X_test), digits=4))

#results_ML_Models = pd.DataFrame()
results_data_3_resampled_NB = pd.DataFrame({'Model':['MultinomialNB Classifier'], 'accuracy': [metrics.accuracy_score(y_test, y_pred_NB)]},index={'2'})
results_ML_Models = pd.concat([results_ML_Models, results_data_3_resampled_NB])
results_ML_Models

from sklearn.naive_bayes import ComplementNB

data_3_resampled_CNB = ComplementNB()
data_3_resampled_CNB.fit(X_train, y_train)

y_pred_CNB = data_3_resampled_CNB.predict(X_test)

accuracy_score(y_test,y_pred_CNB)
print(classification_report(y_test, data_3_resampled_CNB.predict(X_test), digits=4))

#results_ML_Models = pd.DataFrame()
results_data_3_resampled_CNB = pd.DataFrame({'Model':['ComplementNB Classifier'], 'accuracy': [metrics.accuracy_score(y_test, y_pred_CNB)]},index={'2'})
results_ML_Models = pd.concat([results_ML_Models, results_data_3_resampled_CNB])
results_ML_Models


Number of Samples: 46991
Number of Labels:  46991
Number of Training Samples: 37592
Number of Validation Samples: 9399
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000       141
           1     0.4528    0.2017    0.2791       119
           2     0.0149    0.0075    0.0100       133
           3     0.1786    0.1172    0.1415       128
           4     1.0000    0.0073    0.0145       137
           5     0.0364    0.0163    0.0225       123
           6     0.0000    0.0000    0.0000       118
           7     0.3833    0.1679    0.2335       137
           8     0.4000    0.0292    0.0544       137
           9     0.1244    0.1931    0.1514       145
          10     0.0000    0.0000    0.0000       123
          11     0.0000    0.0000    0.0000       121
          12     0.0000    0.0000    0.0000       111
          13     0.8333    0.0847    0.1538       118
          14     0.2553    0.0909    0.1341       132
          15    

,Model,accuracy
1,Random Forest,0.949463
1,Random Forest Classifier - Weighted,0.949782
1,Adaboost Classifier,0.082349
1,Bagging Classifier,0.947016
1,GradientBoost Classifier,0.874455
1,XGBoost Classifier,0.882222
1,CatBoost Classifier,0.931908
2,SVM Classifier,0.279817
2,SGD Classifier,0.274391
2,MultinomialNB Classifier,0.299394


# ML Model 8: KNN (Dataset 3)

In [25]:
tokenizer,X = wordTokenizer(data_3_resampled[data_3_resampled.columns[1]])

y = np.asarray(data_3_resampled[data_3_resampled.columns[4]])
X = pad_sequences(X, maxlen = maxlen)

print("Number of Samples:", len(X))
print("Number of Labels: ", len(y))


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

print("Number of Training Samples:", len(X_train))
print("Number of Validation Samples:", len(X_test))

from sklearn.neighbors import KNeighborsClassifier
scores =[]
# for k in range(1,50):
#     NNH = KNeighborsClassifier(n_neighbors = k, weights = 'uniform', metric='euclidean')
#     NNH.fit(X_train, y_train)
#     scores.append(NNH.score(X_test, y_test))
    
# plt.plot(range(1,50),scores)

data_3_resampled_KNN = KNeighborsClassifier(n_neighbors= 12 , weights = 'uniform', metric='euclidean')

start = time.time()
data_3_resampled_KNN.fit(X_train,y_train)
time.sleep(1)
end = time.time()
print(f"Time Taken To Train KNN on dataset 3 : {end - start}")

y_pred=data_3_resampled_KNN.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

print(classification_report(y_test, data_3_resampled_KNN.predict(X_test), digits=4))

#results_ML_Models = pd.DataFrame()
results_data_3_resampled_KNN= pd.DataFrame({'Model':['KNN Classifier'], 'accuracy': [metrics.accuracy_score(y_test, y_pred)]},index={'1'})
results_ML_Models  = pd.concat([results_ML_Models, results_data_3_resampled_KNN])
results_ML_Models

Number of Samples: 46991
Number of Labels:  46991
Number of Training Samples: 37592
Number of Validation Samples: 9399
Time Taken To Train KNN on dataset 3 : 2.081286907196045
Accuracy: 0.8107245451643792
              precision    recall  f1-score   support

           0     0.5714    0.1135    0.1893       141
           1     0.8548    0.8908    0.8724       119
           2     0.6625    0.3985    0.4977       133
           3     0.8153    1.0000    0.8982       128
           4     0.3600    0.1314    0.1925       137
           5     0.4154    0.2195    0.2872       123
           6     0.5354    0.4492    0.4885       118
           7     0.8671    1.0000    0.9288       137
           8     0.6220    0.5766    0.5985       137
           9     0.8683    1.0000    0.9295       145
          10     0.5682    0.6098    0.5882       123
          11     0.3833    0.1901    0.2541       121
          12     0.4118    0.1892    0.2593       111
          13     0.8194    1.0000    0

,Model,accuracy
1,Random Forest,0.949463
1,Random Forest Classifier - Weighted,0.949782
1,Adaboost Classifier,0.082349
1,Bagging Classifier,0.947016
1,GradientBoost Classifier,0.874455
1,XGBoost Classifier,0.882222
1,CatBoost Classifier,0.931908
2,SVM Classifier,0.279817
2,SGD Classifier,0.274391
2,MultinomialNB Classifier,0.299394


# ML Model 8: Logestic Regression (Dataset 3)

In [26]:
tokenizer,X = wordTokenizer(data_3_resampled[data_3_resampled.columns[1]])

y = np.asarray(data_3_resampled[data_3_resampled.columns[4]])
X = pad_sequences(X, maxlen = maxlen)

print("Number of Samples:", len(X))
print("Number of Labels: ", len(y))


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

print("Number of Training Samples:", len(X_train))
print("Number of Validation Samples:", len(X_test))

from sklearn.linear_model import LogisticRegression

data_3_resampled_Log = LogisticRegression()

start = time.time()
data_3_resampled_Log.fit(X_train,y_train)
time.sleep(1)
end = time.time()
print(f"Time Taken To Train Logistic  on dataset 3 : {end - start}")

y_pred = data_3_resampled_Log.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

print(classification_report(y_test, data_3_resampled_Log.predict(X_test), digits=4))

#results_ML_Models = pd.DataFrame()
results_data_3_resampled_Log= pd.DataFrame({'Model':['Logistic Regression'], 'accuracy': [metrics.accuracy_score(y_test, y_pred)]},index={'1'})
results_ML_Models  = pd.concat([results_ML_Models, results_data_3_resampled_Log])
results_ML_Models

Number of Samples: 46991
Number of Labels:  46991
Number of Training Samples: 37592
Number of Validation Samples: 9399
Time Taken To Train Logistic  on dataset 3 : 32.95309853553772
Accuracy: 0.41791679965953826
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000       141
           1     0.4336    0.4118    0.4224       119
           2     0.6087    0.3158    0.4158       133
           3     0.6049    0.3828    0.4689       128
           4     0.1475    0.0657    0.0909       137
           5     0.3158    0.1463    0.2000       123
           6     0.2593    0.0593    0.0966       118
           7     0.4444    0.3504    0.3918       137
           8     0.2361    0.1241    0.1627       137
           9     0.2083    0.2759    0.2374       145
          10     0.3208    0.1382    0.1932       123
          11     0.0930    0.0331    0.0488       121
          12     0.0597    0.1081    0.0769       111
          13     0.6500    0.44

,Model,accuracy
1,Random Forest,0.949463
1,Random Forest Classifier - Weighted,0.949782
1,Adaboost Classifier,0.082349
1,Bagging Classifier,0.947016
1,GradientBoost Classifier,0.874455
1,XGBoost Classifier,0.882222
1,CatBoost Classifier,0.931908
2,SVM Classifier,0.279817
2,SGD Classifier,0.274391
2,MultinomialNB Classifier,0.299394


# Summary

 
- Model Development using Machine Learning models. We have explored 13 machine learning models in Part 5/6 of the source code. The performance is also evaluation. (please refer accuracies, performacne evaluation from above or from interim report for more details): 

- Machine Learning Models 

| Model | Test accuracy |
| :- | -: |
| Random Forest | 0.949463
| Random Forest Classifier - Weighted | 0.949782
| Adaboost Classifier | 0.082349
| Bagging Classifier | 0.947016
| GradientBoost Classifier | 0.874455
| XGBoost Classifier | 0.882222
| CatBoost Classifier | 0.931908
| SVM Classifier | 0.279817
| SGD Classifier | 0.274391
| MultinomialNB Classifier | 0.299394
| ComplementNB Classifier | 0.107245
| KNN Classifier | 0.810725
| Logistic Regression | 0.417917

Part 6/6: Part 6/6 focuses on HyperParameter Tuning for the models explored. Random Forest model seems to be performing well comared to other classifiers. Badding classifier is also performing equivalently.